# Part 1: ETL Pipeline for Preprocessing the Files 

In [1]:
# import python packages
import os
import re 
import csv
import glob
import json
import cassandra
import pandas as pd
import numpy as np


### Creating a list of filepaths to process original event csv data files

In [2]:
# check your current working directory
print(os.getcwd())

/Users/itsmuriuki/Desktop/Data_Engineering/Data_Modeling/project_2_NoSQL_Apache_Cassandra


In [3]:
#get your current folder and sub folder event data 
filepath = os.getcwd() + '/event_data'
filepath

'/Users/itsmuriuki/Desktop/Data_Engineering/Data_Modeling/project_2_NoSQL_Apache_Cassandra/event_data'

In [4]:
# create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    #join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root, '*'))
    print(file_path_list)

['/Users/itsmuriuki/Desktop/Data_Engineering/Data_Modeling/project_2_NoSQL_Apache_Cassandra/event_data/2018-11-15-events.csv', '/Users/itsmuriuki/Desktop/Data_Engineering/Data_Modeling/project_2_NoSQL_Apache_Cassandra/event_data/2018-11-22-events.csv', '/Users/itsmuriuki/Desktop/Data_Engineering/Data_Modeling/project_2_NoSQL_Apache_Cassandra/event_data/2018-11-09-events.csv', '/Users/itsmuriuki/Desktop/Data_Engineering/Data_Modeling/project_2_NoSQL_Apache_Cassandra/event_data/2018-11-18-events.csv', '/Users/itsmuriuki/Desktop/Data_Engineering/Data_Modeling/project_2_NoSQL_Apache_Cassandra/event_data/2018-11-04-events.csv', '/Users/itsmuriuki/Desktop/Data_Engineering/Data_Modeling/project_2_NoSQL_Apache_Cassandra/event_data/2018-11-01-events.csv', '/Users/itsmuriuki/Desktop/Data_Engineering/Data_Modeling/project_2_NoSQL_Apache_Cassandra/event_data/2018-11-27-events.csv', '/Users/itsmuriuki/Desktop/Data_Engineering/Data_Modeling/project_2_NoSQL_Apache_Cassandra/event_data/2018-11-10-even

### Processing the files to create the data file csv that will be used for apache cassandra tables 

In [5]:
#initiating an empty list of rows that will be generated from each file 
full_data_rows_list = []

# for every filepath in the file path list 
for f in file_path_list:
    
    #read csv file
    with open(f,'r', encoding='utf8', newline='') as csvfile:
        #creating a csv reader object
        csvreader = csv.reader(csvfile)
        next (csvreader)
        
        #extracting each data row, one by one and appending it
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line)
            
#getting the total number of rows
print(len(full_data_rows_list))

8056


In [6]:
# looking at list of event data 
# print(full_data_rows_list)

### Creating one event_datafile_new.csv file to write from

In [7]:
# Creating a smaller event data csv file called event_datafile_ful.csv that will be used to insert data into the 
# Apache casandra tables 
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding='utf8', newline='') as f:
    writer= csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [8]:
#check the number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding='utf8') as f:
    print(sum(1 for line in f))

6821


# Part 2. Coding the Apache Cassandra part of things

We are now going to work with the *event_datafile_new.csv* 

In [9]:
#let's have a look at it 
df = pd.read_csv('event_datafile_new.csv')

In [10]:
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Harmonia,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",583,Sehr kosmisch,26
1,The Prodigy,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",583,The Big Gundown,26
2,Train,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",583,Marry Me,26
3,Sony Wonder,Samuel,M,0,Gonzalez,218.06975,free,"Houston-The Woodlands-Sugar Land, TX",597,Blackbird,61
4,Van Halen,Tegan,F,2,Levine,289.38404,paid,"Portland-South Portland, ME",602,Best Of Both Worlds (Remastered Album Version),80


In [11]:
#the columns in the event_datafile_new.csv
df.columns

Index(['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length',
       'level', 'location', 'sessionId', 'song', 'userId'],
      dtype='object')

### Writing the Apache Cassandra code

#### creating a cluster

In [12]:
from cassandra.cluster import Cluster
# from cassandra.policies import DCAwareRoundRobinPolicy
# from cassandra.auth import PlainTextAuthProvider

#connecting to local cluster
try:
    cluster = Cluster()
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)


# credentials would be different when connecting to a different instance.
# def cassandra_conn():

#    auth_provider = PlainTextAuthProvider(username='cassandra', password='password')
#    cluster = Cluster(['127.0.0.1'], load_balancing_policy=DCAwareRoundRobinPolicy(local_dc='US-WEST'), port=9042, auth_provider=auth_provider)

#    session = cluster.connect()

#    return session, cluster



#### Create Keyspace

In [13]:
create_keyspace_command = """CREATE KEYSPACE IF NOT EXISTS sparkify
                             WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""

try:
    session.execute(create_keyspace_command)
except Exception as e:
    print(e)

#### Set Keyspace

In [14]:
set_keyspace_command = "USE sparkify"

try:
    session.execute(set_keyspace_command)
except Exceptionption as e:
    print(e)



# part 3: Modelling the database on the queries given

Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.



Create queries to ask the following three questions of the data
1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

## Query 1

In [15]:
#1. Give me the artist, song title and song's length in the music app history that was heard during 
#sessionId = 338, and itemInSession = 4

# we will use seessionId and itemSession
# Our complete PRIMARY KEY is composed by session_id, session_item_number

create_statement = """CREATE TABLE IF NOT EXISTS song_data_by_session(session_id INT,
                    session_item_number INT,
                    artist_name TEXT,
                    song_title TEXT,
                    song_length DOUBLE,
                    PRIMARY KEY((session_id, session_item_number))) """

session.execute(create_statement)

select_statement = "SELECT artist_name, song_title, song_length FROM song_data_by_session WHERE session_id = %s AND session_item_number = %s"


In [16]:
#setting up the csv file 
file = 'event_datafile_new.csv'

with open(file, encoding= 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) #skip header
    for line in csvreader:
        query = """INSERT INTO song_data_by_session(session_id, session_item_number, artist_name, song_title, song_length)"""
        query= query + "VALUES (%s, %s, %s, %s, %s )"
        
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data has been inserted into each table

In [17]:
# Add in the SELECT statement to verify the data was entered into the table
result_set = session.execute(select_statement, (338, 4))

for row in result_set:
    print(row)

Row(artist_name='Faithless', song_title='Music Matters (Mark Knight Dub)', song_length=495.3073)


## Query 2

copy and paste the above three cells for the rest of the queries 

In [20]:
#2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)
# for userid = 10, sessionid = 182


## In this case user_id and session_id are the COMPOUND PARTITION KEY  \
## this allows us to have a unique PRIMARY KEY for our query, but for this request we have to \
## order by session_item_number but not to query on that so we have to declare session_item_number as CLUSTERING KEY. \
## Our complete PRIMARY KEY is composed by user_id, session_id, session_item_numbe

create_statement = """CREATE TABLE IF NOT EXISTS song_user_data_by_user_and_session_data(user_id INT,
                    session_id INT,
                    session_item_number INT,
                    artist_name TEXT,
                    song_title TEXT,
                    user_first_name TEXT,
                    user_last_name TEXT,
                    PRIMARY KEY ((user_id,session_id), session_item_number))"""

session.execute(create_statement)

select_statement = """SELECT artist_name, song_title, song_length FROM song_user_data_by_user_and_session_data WHERE user_id = %s AND session_id = %s"""

In [21]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next (csvreader) #skip header
    for line in csvreader:
        
        query = """INSERT INTO song_user_data_by_user_and_session_data(user_id, session_id, session_item_number, artist_name, song_title,
                user_first_name, user_last_name)"""
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[4], line[4]))
        
result_set = session.execute(select_statement, (10, 182))

for row in result_set:
    print(row)

InvalidRequest: Error from server: code=2200 [Invalid query] message="Undefined column name song_length"

## Query 3

In [ ]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

## In this case, song_title is the PARTITION KEY and user_id \
## is the CLUSTERING KEY, the request asks to retrieve the user name \
## by song title, so we have to set song_title as PARTITION KEY, but \
## more users can listen to the same song so we may have many INSERT with the \
## same key, Cassandra overwrites data with the same key so we need to add a CLUSTERING KEY \
## because we need to have a unique record but not to query on that. \
## Our complete PRIMARY KEY is composed by song_title, user_id 

create_statement = """CREATE TABLE IF NOT EXISTS user_data_by_song_title (song_title TEXT,
                    user_id INT,
                    user_first_name TEXT,
                    user_last_name TEXT,
                    PRIMARY KEY((song_title), user_id))"""

session.execute(create_statement)

with open (file, encoding ='utf8') as f:
    csvreader= csv.reader(f)
    next (csvreader) #skip header
    for line in csvreader:
        
        query = """INSERT INTO user_data_by_song_title(song_title, user_id, user_first_name, user_last_name )"""
        query = query + "VALUES (%s, %s, %s, %s )"

        session.execute(query, (line[9], int(line[10]), line[1], line[4]))
        
select_statement = """SELECT user_first_name, user_last_name FROM user_data_by_song_title WHERE song_title= %s """
        
result_set = session.execute(select_statement, ('All Hands Against His Own', ))

for row in result_set:
    print(row)

### Drop the tables before closing out the sessions 

In [ ]:
drop_song_data_by_session = """DROP TABLE song_data_by_session"""
drop_song_user_data_by_user_and_session_data = """DROP TABLE song_user_data_by_user_and_session_data"""
drop_user_data_by_song_title = """DROP TABLE user_data_by_song_title """

In [ ]:
session.execute(drop_song_data_by_session)
session.execute(drop_song_user_data_by_user_and_session_data)
session.execute(drop_user_data_by_song_title)

### Close the session and cluster connection 


In [ ]:
session.shutdown()
cluster.shutdown()

In [ ]:
print('Process Ended')